In [217]:
# Zipcodes to ZCTAs mapping
# Socio-economic KNN classification

from pandas import read_csv, merge, read_excel, ExcelFile, concat



In [218]:
#Read Data
zctas = read_csv('.\Input\ZCTA_Household_income.csv')

with ExcelFile(r'.\Input\ZiptoZcta_Crosswalk_2021.xlsx') as xl :
    df = concat([read_excel(xl, sheet_name = s ) for s in xl.sheet_names],
                ignore_index= True)


In [219]:
df.ZCTA = df.ZCTA.astype('str')

print(f'df.ZIP_CODE dtype: {df.ZIP_CODE.dtype}, with {df.ZIP_CODE.nunique()} unique values.')
print(f'df.ZCTA dtype: {df.ZCTA.dtype}, with {df.ZCTA.nunique()} unique values.')

df.ZIP_CODE dtype: int64, with 41107 unique values.
df.ZCTA dtype: object, with 33144 unique values.


In [220]:
#Check the length of ZCTA codes
df['len'] = df.ZCTA.str.len()
df['len'].unique() # There are  5, nan and 7 length codes
df[df['len'] == 7] # This represent 'No ZCTA' drop	
df.drop(df[df['len'] == 7].index, inplace= True)

df.ZCTA = [c if len(c) == 5 else '0'+ c if len(c) == 4 else '00'+c 
             for c in df.ZCTA]

In [221]:
print(f'df.ZIP_CODE dtype: {df.ZIP_CODE.dtype}, with {df.ZIP_CODE.nunique()} unique values.')
print(f'df.ZCTA dtype: {df.ZCTA.dtype}, with {df.ZCTA.nunique()} unique values.')

df.ZIP_CODE dtype: int64, with 41099 unique values.
df.ZCTA dtype: object, with 33139 unique values.


In [222]:
zctas.head()

,ZCTA,Households100to149K,Households150to199K,Households200KorMore,RFM1,RFM2,RFM3
0,35007,2224,473,254,9,8,7
1,35124,2012,809,805,9,8,8
2,35173,2265,1037,980,9,9,9
3,35215,1896,434,264,9,8,7
4,35216,2065,940,1702,9,9,9


In [223]:
zctas.ZCTA = zctas.ZCTA.astype('str')
print(f'ZCTA dtype: {zctas.ZCTA.dtype} with {zctas.ZCTA.nunique()} unique values')

# Convert all ZCTAs to 5 digit
zctas.ZCTA = [c if len(c) == 5 else '0'+ c if len(c) == 4 else '00'+c 
             for c in zctas.ZCTA]


ZCTA dtype: object with 33120 unique values


In [224]:
df = df.iloc[:, [0,2,4]].copy()
df = df.drop_duplicates(keep='last')

df = df.merge(zctas, how='right', left_on= 'ZCTA', right_on= 'ZCTA')


In [225]:
#convert zip_codes to 5-digit str object
df.ZIP_CODE = df.ZIP_CODE.astype('Int64') 
df.drop(df[df.ZIP_CODE.isna()].index, inplace= True) # drop rows with missing zipcodes
df.ZIP_CODE = df.ZIP_CODE.astype('str')
df.ZIP_CODE = [c if len(c) == 5 else '0'+ c if len(c) == 4 else '00'+c 
             for c in df.ZIP_CODE]
df